## Module 3 Assignment - Building RAG systems with a Vector Database

---

In this assignment you will work with [Weaviate API](https://weaviate.io/) to build a tiny RAG system. You will:

- Load a [collection](https://weaviate.io/developers/weaviate/manage-data/collections) for BBC News data.
- Use the Weaviate API to retrieve documents from the vector database.
- Create functions to retrieve data based on Semantic Search, BM25 and Hybrid Search (using RRF) using the Weaviate API.
- Use an LLM to generate responses.

**IMPORTANT**: This assignment assumes you know how to handle simple tasks with collections in the Weaviate API. If you are not familiar with it yet, please read the Ungraded Lab on the Weaviate API! Furthermore, the data you will be working here is already *chunked*. You can get more hands-on experience on chunking reading the Ungraded Lab on chunking!


# Table of Contents
- [ 1 - Loading the libraries](#1)
- [ 2 - Setting up the Weaviate Client and loading the data](#2)
  - [ 2.1 Loading the Weaviate Client](#2-1)
  - [ 2.2 Loading the data](#2-2)
- [ 3 - Loading the Collection](#3)
  - [ 3.1 Metadata filtering](#3-1)
    - [ Exercise 1](#ex01)
  - [ 3.2 Semantic search](#3-2)
    - [ Exercise 2](#ex02)
  - [ 3.3 BM25 Serach](#3-3)
    - [ Exercise 3](#ex03)
  - [ 3.4 Hybrid search](#3-4)
    - [ Exercise 4](#ex04)
    - [ Exercise 5](#ex05)
- [ 4 - Incorporating the Weaviate API into our previous schema](#4)
  - [ 4.1 Generating the final prompt](#4-1)
  - [ 4.2 LLM call](#4-2)
- [ 5 - Experimenting with Your RAG System](#5)


---
<h4 style="color:black; font-weight:bold;">USING THE TABLE OF CONTENTS</h4>

JupyterLab provides an easy way for you to navigate through your assignment. It's located under the Table of Contents tab, found in the left panel, as shown in the picture below.

![TOC Location](images/toc.png)

---

<h4 style="color:green; font-weight:bold;">TIPS FOR SUCCESSFUL GRADING OF YOUR ASSIGNMENT:</h4>

- All cells are frozen except for the ones where you need to submit your solutions or when explicitly mentioned you can interact with it.

- You can add new cells to experiment but these will be omitted by the grader, so don't rely on newly created cells to host your solution code, use the provided places for this.

- Avoid using global variables unless you absolutely have to. The grader tests your code in an isolated environment without running all cells from the top. As a result, global variables may be unavailable when scoring your submission. Global variables that are meant to be used will be defined in UPPERCASE.

- - To submit your notebook for grading, first save it by clicking the 💾 icon on the top left of the page and then click on the <span style="background-color: blue; color: white; padding: 3px 5px; font-size: 16px; border-radius: 5px;">Submit assignment</span> button on the top right of the page.
---

<a id='1'></a>
## 1 - Loading the libraries

---

Run the cell below to load the necessary libraries for this assignment.

In [9]:
from typing import Optional, List, Dict, Callable

import joblib
import weaviate
from weaviate.classes.query import Filter, Rerank
from weaviate.classes.config import Configure, Property, DataType
import tqdm

In [2]:
from utils import (
    generate_with_single_input,
    print_object_properties,
    display_widget,
    kill_processes_on_ports,
)
import unittests

kill_processes_on_ports([5000])

import flask_app

c:\Users\91811\AppData\Local\pypoetry\Cache\virtualenvs\rag-kkX-x6bK-py3.13\Lib\site-packages\together\types\evaluation.py:78: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class EvaluationJob(BaseModel):
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute


 * Serving Flask app 'flask_app'
 * Debug mode: off


<a id='2'></a>
## 2 - Setting up the Weaviate Client and loading the data

---

In this section, you will set up the Weaviate client and load the data, which consists of the [BBC news dataset](https://www.kaggle.com/datasets/gpreda/bbc-news) adapted from Kaggle.

<a id='2-1'></a>
### 2.1 Loading the Weaviate Client

Let's connect the Weaviate client to begin working with the Weaviate API. The server is already running on the backend. 

**Troubleshooting:**

- If you encounter issues loading the next cell, try restarting your kernel, clicking in the circled arrow in the panel above.

In [3]:
client = weaviate.connect_to_local()

<a id='2-2'></a>
### 2.2 Loading the data

Now, let's load the data. The dataset is structured with the following fields:

- **`title`**: The headline of the article.
- **`pubDate`**: The publication date and time of the article.
- **`guid`**: A unique identifier for the article, commonly used for listing.
- **`link`**: A URL link to access the full article online.
- **`description`**: A brief summary or teaser of the article's content.
- **`article_content`**: The complete text of the article, providing detailed information.

In [5]:
bbc_data = joblib.load("data/bbc_data.joblib")

In [11]:
print(f"{len(bbc_data)} news articles")
print("Sample:")
print_object_properties(bbc_data[0])

9973 news articles
Sample:
article_content: Justin Welby speaks on BBC Radio 4's Today programme as part of a special show guest edited by Dame ...(truncated)
description: The Archbishop of Canterbury urges politicians to "forswear wedge issues" and avoid divisive topics.
guid: https://www.bbc.co.uk/news/uk-67844356
link: https://www.bbc.co.uk/news/uk-67844356?at_medium=RSS&at_campaign=KARANGA
pubDate: 2024-01-01 00:00:04
title: Justin Welby: Political leaders should treat opponents as human beings



<a id='3'></a>
## 3 - Loading the Collection

---

In this section, you will load the collection containing the BBC News dataset.

In [4]:
if not client.collections.exists(
    "bbc_collection"
):  # Creates only if the collection does not exist
    collection = client.collections.create(
        name="bbc_collection",
        vectorizer_config=[
            Configure.NamedVectors.text2vec_transformers(
                name="main_vector",  # This is the name you will need to access the vectors of the objects in your collection
                source_properties=[
                    "title",
                    "chunk",
                ],  # which properties should be used to generate a vector, they will be appended to each other when vectorizing
                vectorize_collection_name=False,  # This tells the client to not vectorize the collection name.
                # If True, it will be appended at the beginning of the text to be vectorized
            )
        ],
        reranker_config=Configure.Reranker.transformers(),  # The reranker config
        properties=[  # Define properties
            Property(
                name="title", vectorize_property_name=False, data_type=DataType.TEXT
            ),
            Property(
                name="chunk", vectorize_property_name=False, data_type=DataType.TEXT
            ),
            Property(
                name="pubDate",
                vectorize_property_name=False,
                data_type=DataType.DATE,
            ),
            Property(
                name="description",
                vectorize_property_name=False,
                data_type=DataType.TEXT,
            ),
            Property(
                name="article_content",
                vectorize_property_name=False,
                data_type=DataType.TEXT,
            ),
            Property(
                name="link", vectorize_property_name=False, data_type=DataType.TEXT
            ),
            Property(name="chunk_index", data_type=DataType.INT),
        ],
    )
else:
    collection = client.collections.get("bbc_collection")

In [7]:
# Populate the collectios
all_exported_objects = joblib.load("data/bbc_collection_weaviate_export.joblib")

In [9]:
if len(collection) == 0:
    with collection.batch.fixed_size(batch_size=1024, concurrent_requests=1) as batch:
        for chunk_obj in tqdm.tqdm(all_exported_objects["objects"]):
            batch.add_object(properties=chunk_obj)

  0%|          | 0/75256 [00:00<?, ?it/s]

 26%|██▌       | 19503/75256 [09:40<32:02, 29.01it/s]{'message': 'Failed to send 1 in a batch of 1024', 'errors': {'send POST request: Post "http://host.docker.internal:5000//vectors": dial tcp 192.168.65.254:5000: i/o timeout'}}
{'message': 'Failed to send 1 objects in a batch of 1024. Please inspect client.batch.failed_objects or collection.batch.failed_objects for the failed objects.'}
 88%|████████▊ | 65875/75256 [35:53<04:41, 33.27it/s]{'message': 'Failed to send 1 in a batch of 1024', 'errors': {'send POST request: Post "http://host.docker.internal:5000//vectors": dial tcp 192.168.65.254:5000: i/o timeout'}}
{'message': 'Failed to send 1 objects in a batch of 1024. Please inspect client.batch.failed_objects or collection.batch.failed_objects for the failed objects.'}
100%|██████████| 75256/75256 [41:15<00:00, 30.40it/s]


In [13]:
# Get the failed objects
failed = collection.batch.failed_objects

if failed:
    print(f"Retrying {len(failed)} failed objects...")

    # Retry with a smaller batch size and lower concurrency
    with collection.batch.fixed_size(batch_size=1, concurrent_requests=1) as batch:
        for error_obj in failed:
            batch.add_object(
                properties=error_obj.object_.properties, uuid=error_obj.object_.uuid
            )

    # Check if any still failed
    if collection.batch.failed_objects:
        print(f"Still failing: {len(collection.batch.failed_objects)} objects")
    else:
        print("All retries succeeded!")

Retrying 2 failed objects...
All retries succeeded!


In [5]:
print(f"The number of elements in the collection is: {len(collection)}")

The number of elements in the collection is: 75256


Let's fetch one example of object in this collection.

In [6]:
object = collection.query.fetch_objects(limit=1, include_vector=True).objects[0]
print("Printing the properties (some will be truncated due to size)")
print_object_properties(object.properties)
print("Vector: (truncated)", object.vector["main_vector"][0:15])
print("Vector length: ", len(object.vector["main_vector"]))

Printing the properties (some will be truncated due to size)
article_content: Marc Guehi was one of four Crystal Palace players in England's Euro 2024 squad, including Dean Hende...(truncated)
chunk: of interest to champions Manchester City last season but they signed Matheus Nunes instead. Parish s...(truncated)
chunk_index: 4
description: With Newcastle bidding for Crystal Palace defender Marc Guehi, Eagles chairman Steve Parish says it will take "superstar money" to buy him.
link: https://www.bbc.com/sport/football/articles/cy8xp6jlgp1o
pubDate: 2024-08-13 18:51:40+00:00
title: Guehi will cost 'superstar money' - Palace chairman

Vector: (truncated) [-0.04248927906155586, -0.03243854269385338, 0.04636770486831665, 0.001350492238998413, 0.03626907244324684, -0.017493968829512596, 0.043198294937610626, 0.04196828976273537, 0.009826821275055408, -0.04336152598261833, -0.003917561378329992, 0.003330694977194071, -0.04791957885026932, 0.036959242075681686, 0.03621198982000351]
Vector len

The vector length is `768`. So every chunk in the vector database is mapped into a 768 dimension vector. This is the vector the Weaviate API uses to perform semantic search.

<a id='ex01'></a>

<a id='3-1'></a>
### 3.1 Metadata filtering

<a id='ex01'></a>
### Exercise 1

In this exercise, you will implement a metadata filtering function. This function will take several inputs: a property (such as `article_content`, `title`, `pubDate`, etc.), the values you want to filter by, the collection you want to search in, and the number of items you want to retrieve.

<details>
<summary style="color: green;">Hint 1</summary>
<p>Remember that to perform filtering based only on metadata, the appropriate method to use is <code>collection.query.fetch_objects</code>.</p>
</details>
<details>
<summary style="color: green;">Hint 2</summary>
<p>When using <code>collection.query.fetch_objects</code>, you must provide the <code>metadata_property</code> as the <code>property</code> and the corresponding <code>Filter</code> object.</p>
</details>
<details>
<summary style="color: green;">Hint 3</summary>
<p>The filter object should be used with the method <code>.by_property</code> for the appropriate property, and <code>.contains_any</code> with the relevant values. A typical call would be <code>Filter.by_property(metadata_property).contains_any(values)</code>.</p>
<p>To limit the results, use <code>limit=limit</code> within the <code>.fetch_objects</code> method.</p>
</details>

In [7]:
# GRADED CELL


def filter_by_metadata(
    metadata_property: str,
    values: list[str],
    collection: weaviate.collections.Collection,
    limit: int = 5,
) -> list:
    """
    Retrieves objects from a specified collection based on metadata filtering criteria.

    This function queries a collection within the specified client to fetch objects that match
    certain metadata criteria. It uses a filter to find objects whose specified 'property' contains
    any of the given 'values'. The number of objects retrieved is limited by the 'limit' parameter.

    Args:
    metadata_property (str): The name of the metadata property to filter on.
    values (List[str]): A list of values to be matched against the specified property.
    collection_name (weaviate.collections.collection.sync.Collection): The collection to query.
    limit (int, optional): The maximum number of objects to retrieve. Defaults to 5.

    Returns:
    List[Object]: A list of objects from the collection that match the filtering criteria.
    """
    ### START CODE HERE ###

    # Retrieve using collection.query.fetch_objects

    response = collection.query.fetch_objects(
        limit=limit,
        filters=Filter.by_property(metadata_property).contains_any(values),
    )

    ### END CODE HERE ###

    response_objects = [x.properties for x in response.objects]

    return response_objects

In [8]:
# Let's get an example
res = filter_by_metadata("title", ["Taylor Swift"], collection, limit=2)
for x in res:
    print_object_properties(x)

article_content: Three upcoming Taylor Swift concerts in Vienna have been cancelled following a possible security thr...(truncated)
chunk: was making friendship bracelets - which fans have been taking to the shows to swap with each other d...(truncated)
chunk_index: 4
description: The cancellations follow the arrest of two people suspected of planning attacks.
link: https://www.bbc.com/news/articles/ce31zxqypxpo
pubDate: 2024-08-08 00:00:58+00:00
title: Taylor Swift Vienna concerts cancelled after attack threat

article_content: Taylor Swift surprised fans with a double album on Friday but dropped no singles in advance BeyoncÃ©...(truncated)
chunk: can be a single - and anything can enter the chart. "The great thing about the digital environment i...(truncated)
chunk_index: 4
description: The two megastars have both ditched singles for their latest releases - will others follow their lead?
link: https://www.bbc.co.uk/news/newsbeat-68848506
pubDate: 2024-04-27 23:12:34+00:00
title: Tayl

**Expected output**
```
article_content: The 2024 awards season kicked off in style at the Golden Globes - the first major red carpet event o...(truncated)
chunk: some of his previous get-ups. The Bear's Jeremy Allen White - who recently became the new face (and ...(truncated)
chunk_index: 4
description: Stars including Margot Robbie and Taylor Swift arrived in a variety of eye-catching outfits.
link: https://www.bbc.co.uk/news/entertainment-arts-67908727?at_medium=RSS&at_campaign=KARANGA
pubDate: 2024-01-08 03:23:58+00:00
title: Margot Robbie, Taylor Swift and more on Golden Globes red carpet

article_content: The 2024 awards season kicked off in style at the Golden Globes - the first major red carpet event o...(truncated)
chunk: headpiece - not entirely a fashion choice. She says the "protective veil" is because she hurt her fa...(truncated)
chunk_index: 5
description: Stars including Margot Robbie and Taylor Swift arrived in a variety of eye-catching outfits.
link: https://www.bbc.co.uk/news/entertainment-arts-67908727?at_medium=RSS&at_campaign=KARANGA
pubDate: 2024-01-08 03:23:58+00:00
title: Margot Robbie, Taylor Swift and more on Golden Globes red carpet
```

In [9]:
# Test your solution!
unittests.test_filter_by_metadata(filter_by_metadata, client)

 All tests passed!


<a id='ex02'></a>

<a id='3-2'></a>
### 3.2 Semantic search

<a id='ex02'></a>
### Exercise 2

In this exercise, you will implement a semantic search retrieval, similar to the one you created in the previous assignment, but this time utilizing the Weaviate API.

<details>
<summary style="color: green;">Hint</summary>
<p>Remember that to perform semantic search, you should use the method <code>collection.query.near_text</code>.</p>
<p>The <code>top_k</code> parameter in the function dictates how many results to retrieve. In Weaviate, this is referred to as <code>limit</code>. Adjust this parameter as needed.</p>
</details>

In [12]:
# GRADED CELL


def semantic_search_retrieve(
    query: str, collection: weaviate.collections.Collection, top_k: int = 5
) -> list:
    """
    Performs a semantic search on a collection and retrieves the top relevant chunks.

    This function executes a semantic search query on a specified collection to find text chunks
    that are most relevant to the input 'query'. The search retrieves a limited number of top
    matching objects, as specified by 'top_k'. The function returns the 'chunk' property of
    each of the top matching objects.

    Args:
    query (str): The search query used to find relevant text chunks.
    collection (weaviate.collections.collection.sync.Collection): The collection in which the semantic search is performed.
    top_k (int, optional): The number of top relevant objects to retrieve. Defaults to 5.

    Returns:
    List[str]: A list of text chunks that are most relevant to the given query.
    """
    ### START CODE HERE ###

    # Retrieve using collection.query.near_text
    response = collection.query.near_text(query=query, limit=top_k)

    ### END CODE HERE ###

    response_objects = [x.properties for x in response.objects]

    return response_objects

In [11]:
# Let's have an example!
print_object_properties(
    semantic_search_retrieve(
        query="Tell me about the last Taylor Swift show", collection=collection, top_k=2
    )
)

article_content: Taylor Swift has finished the European leg of her Eras Tour with a record-breaking show at Wembley S...(truncated)
chunk: size crowd at all". At an earlier show in Liverpool, she had also called the Eras Tour the “most exh...(truncated)
chunk_index: 10
description: The star is joined by Florence + The Machine and sings So Long, London at her final UK show.
link: https://www.bbc.com/news/articles/cr5nr3n6epvo
pubDate: 2024-08-21 03:02:08+00:00
title: 'I've never had it this good' - Taylor Swift thanks fans after new Wembley record

article_content: Taylor Swift has finished the European leg of her Eras Tour with a record-breaking show at Wembley S...(truncated)
chunk: regular part of the setlist. Last week, the star was joined by Ed Sheeran to play the songs Endgame ...(truncated)
chunk_index: 4
description: The star is joined by Florence + The Machine and sings So Long, London at her final UK show.
link: https://www.bbc.com/news/articles/cr5nr3n6epvo
pubDate: 2024-08-2

**Expected Output**
```
article_content: Taylor Swift has finished the European leg of her Eras Tour with a record-breaking show at Wembley S...(truncated)
chunk: size crowd at all". At an earlier show in Liverpool, she had also called the Eras Tour the “most exh...(truncated)
chunk_index: 10
description: The star is joined by Florence + The Machine and sings So Long, London at her final UK show.
link: https://www.bbc.com/news/articles/cr5nr3n6epvo
pubDate: 2024-08-21 03:02:08+00:00
title: 'I've never had it this good' - Taylor Swift thanks fans after new Wembley record

article_content: Taylor Swift has finished the European leg of her Eras Tour with a record-breaking show at Wembley S...(truncated)
chunk: regular part of the setlist. Last week, the star was joined by Ed Sheeran to play the songs Endgame ...(truncated)
chunk_index: 4
description: The star is joined by Florence + The Machine and sings So Long, London at her final UK show.
link: https://www.bbc.com/news/articles/cr5nr3n6epvo
pubDate: 2024-08-21 03:02:08+00:00
title: 'I've never had it this good' - Taylor Swift thanks fans after new Wembley record

```

In [12]:
unittests.test_semantic_search_retrieve(semantic_search_retrieve, client)

 All tests passed!


<a id='ex03'></a>

<a id='3-3'></a>
### 3.3 BM25 Serach

<a id='ex03'></a>
### Exercise 3

In this exercise, you will implement a BM25 retrieval, similar to the one you created in the previous assignment, but now using the Weaviate API.
<details>
<summary style="color: green;">Hint</summary>
<p>To perform a BM25 search, use the method <code>collection.query.bm25</code>.</p>
<p>The <code>top_k</code> parameter in the function specifies how many results to retrieve. In Weaviate, this parameter is referred to as <code>limit</code>. Adjust this accordingly.</p>
</details>

In [25]:
# GRADED CELL


def bm25_retrieve(
    query: str, collection: weaviate.collections.Collection, top_k: int = 5
) -> list:
    """
    Performs a BM25 search on a collection and retrieves the top relevant chunks.

    This function executes a BM25-based search query on a specified collection to identify text
    chunks that are most relevant to the provided 'query'. It retrieves a limited number of the
    top matching objects, as specified by 'top_k', and returns the 'chunk' property of these objects.

    Args:
    query (str): The search query used to find relevant text chunks.
    collection (weaviate.collections.collection.sync.Collection): The collection in which the BM25 search is performed.
    top_k (int, optional): The number of top relevant objects to retrieve. Defaults to 5.

    Returns:
    List[str]: A list of text chunks that are most relevant to the given query.
    """

    ### START CODE HERE ###

    # Retrieve using collection.query.bm25
    response = collection.query.bm25(query=query, limit=top_k)

    ### END CODE HERE ###

    response_objects = [x.properties for x in response.objects]
    return response_objects

In [14]:
print_object_properties(
    bm25_retrieve("Tell me about the last Taylor Swift show", collection, top_k=2)
)

article_content: The Tortured Poets Department, released on Friday, features 31 songs For two female journalists in t...(truncated)
chunk: think it just captures a whole spectrum of human emotion where love makes Taylor behave like a kid. ...(truncated)
chunk_index: 13
description: From exes who wasted our time, to comfort-eating after a breakup. We've all been there and so has Swift.
link: https://www.bbc.co.uk/news/entertainment-arts-68856232
pubDate: 2024-04-20 15:36:19+00:00
title: How Taylor Swift captured modern dating despair

article_content: The most lucrative concert tour in music history lands in the UK on Friday, when Taylor Swift plays ...(truncated)
chunk: the resilience and the chutzpah to be the boss of an enormous machine, employing thousands of people...(truncated)
chunk_index: 4
description: As she launches the UK leg of her Eras tour, we look at how Taylor Swift became pop's biggest star.
link: https://www.bbc.com/news/articles/cpvvx9lwl0do
pubDate: 2024-06-07 00:06

**Expected Output**
```
article_content: Rapper Killer Mike won three Grammys in the rap category - best rap song, best rap performance and b...(truncated)
chunk: police brutality and systemic racism. He was a highly visible supporter of Bernie Sanders' two campa...(truncated)
chunk_index: 4
description: The 48-year-old was detained on a misdemeanour charge after winning three awards in the rap category.
link: https://www.bbc.co.uk/news/world-us-canada-68201021?at_medium=RSS&at_campaign=KARANGA
pubDate: 2024-02-05 23:27:08+00:00
title: Killer Mike dismisses arrest at Grammys as 'speed bump'

article_content: Rapper Killer Mike won three Grammys in the rap category - best rap song, best rap performance and b...(truncated)
chunk: Nicki Minaj. He also won a third award for best rap album with his album Michael. "You cannot tell m...(truncated)
chunk_index: 3
description: The 48-year-old was detained on a misdemeanour charge after winning three awards in the rap category.
link: https://www.bbc.co.uk/news/world-us-canada-68201021?at_medium=RSS&at_campaign=KARANGA
pubDate: 2024-02-05 23:27:08+00:00
title: Killer Mike dismisses arrest at Grammys as 'speed bump'
```

In [15]:
unittests.test_bm25_retrieve(bm25_retrieve, client)

 All tests passed!


<a id='ex04'></a>

<a id='3-4'></a>
### 3.4 Hybrid search

<a id='ex04'></a>
### Exercise 4

In this exercise, you will implement a Reciprocal Rank Fusion (RRF) retrieval system using the Weaviate API. To achieve this, you will need to use the `collection.query.hybrid` method.



<details>
<summary style="color: green;">Hint</summary>
<p>To perform a hybrid search, use the method <code>collection.query.hybrid</code>.</p>
<p>The <code>top_k</code> parameter in the function specifies how many results to retrieve. In Weaviate, this is referred to as <code>limit</code>. Make sure to also include the <code>alpha</code>.</p>
</details>

In [22]:
# GRADED CELL


def hybrid_retrieve(
    query: str,
    collection: weaviate.collections.Collection,
    alpha: float = 0.5,
    top_k: int = 5,
) -> list:
    """
    Performs a hybrid search on a collection and retrieves the top relevant chunks.

    This function executes a hybrid search that combines semantic vector search and traditional
    keyword-based search on a specified collection to find text chunks most relevant to the
    input 'query'. The relevance of results is influenced by 'alpha', which balances the weight
    between vector and keyword matches. It retrieves a limited number of top matching objects,
    as specified by 'top_k', and returns the 'chunk' property of these objects.

    Args:
    query (str): The search query used to find relevant text chunks.
    collection (weaviate.collections.collection.sync.Collection): The collection in which the hybrid search is performed.
    alpha (float, optional): A weighting factor that balances the contribution of semantic
    and keyword matches. Defaults to 0.5.
    top_k (int, optional): The number of top relevant objects to retrieve. Defaults to 5.

    Returns:
    List[str]: A list of text chunks that are most relevant to the given query.
    """
    ### START CODE HERE ###

    # Retrieve using collection.query.hybrid
    response = collection.query.hybrid(query=query, alpha=alpha, limit=top_k)

    ### END CODE HERE ###

    response_objects = [x.properties for x in response.objects]

    return response_objects

In [17]:
print_object_properties(
    hybrid_retrieve("Tell me about the last Taylor Swift show", collection, top_k=2)
)

article_content: Taylor Swift was joined by "best friend" Ed Sheeran as she kicked off the final part of the European...(truncated)
chunk: a terror threat. It was also the first time she'd returned to the UK since an attack in Southport la...(truncated)
chunk_index: 1
description: Swift closes the European leg of her tour with five record-breaking performances at Wembley Stadium.
link: https://www.bbc.com/news/articles/cwyw74p8ejeo
pubDate: 2024-08-15 21:27:53+00:00
title: Ed Sheeran joins Taylor Swift for Eras Tour return

article_content: Taylor Swift has won the Grammy for Album of the Year four times - more than any other artist That T...(truncated)
chunk: Taylor Swift's 1989: Her biggest album returns Swift's UK fans lose Â£1m in ticket scams, bank says ...(truncated)
chunk_index: 18
description: The star's 11th album is much more than a break-up record, and may even mark the end of an era.
link: https://www.bbc.co.uk/news/entertainment-arts-68846376
pubDate: 2024-04-19 12:19:09+0

**Expected Output**
```
article_content: Rapper Killer Mike won three Grammys in the rap category - best rap song, best rap performance and b...(truncated)
chunk: police brutality and systemic racism. He was a highly visible supporter of Bernie Sanders' two campa...(truncated)
chunk_index: 4
description: The 48-year-old was detained on a misdemeanour charge after winning three awards in the rap category.
link: https://www.bbc.co.uk/news/world-us-canada-68201021?at_medium=RSS&at_campaign=KARANGA
pubDate: 2024-02-05 23:27:08+00:00
title: Killer Mike dismisses arrest at Grammys as 'speed bump'

article_content: Taylor Swift has finished the European leg of her Eras Tour with a record-breaking show at Wembley S...(truncated)
chunk: size crowd at all". At an earlier show in Liverpool, she had also called the Eras Tour the “most exh...(truncated)
chunk_index: 10
description: The star is joined by Florence + The Machine and sings So Long, London at her final UK show.
link: https://www.bbc.com/news/articles/cr5nr3n6epvo
pubDate: 2024-08-21 03:02:08+00:00
title: 'I've never had it this good' - Taylor Swift thanks fans after new Wembley record
```

In [18]:
unittests.test_hybrid_retrieve(hybrid_retrieve, client)

 All tests passed!


### 5 - Reranking

<a id='ex05'></a>

<a id='ex05'></a>
### Exercise 5

In this section, you will create a new version of `semantic_search` that allows reranking of the results. This new function must support using a different query for reranking or reranking based on a specific document property (e.g., reranking using only the title).

Your task is to add the `rerank` parameter to the `collection.query.near_text` call.

<details>
<summary style="color: green;">Hint 1</summary>
<p>Remember that <code>collection.query.near_text</code> takes a query, a limit (i.e., <code>top_k</code>), and now also requires the <code>rerank</code> parameter.</p>
</details>

<details>
<summary style="color: green;">Hint 2</summary>
<p>The <code>Rerank</code> object is already loaded into memory. It takes two parameters: the query and the document property to use for ranking—<code>query</code> and <code>prop</code>, respectively.</p>
</details>

<details>
<summary style="color: green;">Hint 3</summary>
<p>Define the reranker as <code>reranker = Reranker(appropriate_parameters)</code>. Don’t forget: the query for the reranker should be <code>rerank_query</code>!</p>
</details>

In [5]:
# GRADED CELL


def semantic_search_with_reranking(
    query: str,
    rerank_property: str,
    collection: weaviate.collections.Collection,
    rerank_query: Optional[str] = None,
    top_k: int = 5,
) -> list:
    """
    Performs a semantic search and reranks the results based on a specified property.

    Args:
        query (str): The search query to perform the initial search.
        rerank_property (str): The property used for reranking the search results.
        collection (weaviate.collections.collection.sync.Collection): The collection to search within.
        rerank_query (str, optional): The query to use specifically for reranking. If not provided,
                                      the original query is used for reranking.
        top_k (int, optional): The maximum number of top results to return. Defaults to 5.

    Returns:
        list: A list of properties from the reranked search results, where each item corresponds to
              an object in the collection.
    """
    ### START CODE HERE ###

    # Set the rerank_query to be the same as the query if rerank_query is not passed (don't change this line)
    if rerank_query is None:
        rerank_query = query

    # Define the reranker with rerank_query and rerank_property
    reranker = Rerank(prop=rerank_property, query=rerank_query)

    # Retrieve using collection.query.near_text with the appropriate parameters (do not forget the rerank!)
    response = collection.query.near_text(
        query=query,
        limit=top_k,
        rerank=reranker,
    )

    ### END CODE HERE ###

    response_objects = [x.properties for x in response.objects]

    return response_objects

The reranker model receives a query and a passage (in our case, a chunk of the result) to compute a similarity score.

In [6]:
# Set a query
query = "Tell me about the conflicts in Latin America"
# Get the results from a search (in this case the hybrid search)
results = semantic_search_with_reranking(
    query, collection=collection, top_k=2, rerank_property="chunk"
)

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [7]:
print_object_properties(results)

article_content: A huge diplomatic row has erupted after Spain's transport minister suggested Argentina's president h...(truncated)
chunk: weeks' to attend the launch of Vox's European election campaign, newspaper El Pais reported. Mr Mile...(truncated)
chunk_index: 3
description: A row breaks out after Spain's transport minister suggests Argentina's president has taken drugs.
link: https://www.bbc.com/news/articles/czd8qzvpl4lo
pubDate: 2024-05-04 15:56:45+00:00
title: Spain-Argentina row over drug-use accusation

article_content: Opposition supporters have gathered across Venezuela to protest against Nicolás Maduro's disputed vi...(truncated)
chunk: the world, from Australia to Spain and also in the United Kingdom, Canada, Colombia, Mexico and Arge...(truncated)
chunk_index: 4
description: Opposition leader María Corina Machado joined thousands of demonstrators in the capital Caracas.
link: https://www.bbc.com/news/articles/cgedgqqy7x9o
pubDate: 2024-08-17 23:19:48+00:00
title: Prote

**Expected Results**
```
article_content: A huge diplomatic row has erupted after Spain's transport minister suggested Argentina's president h...(truncated)
chunk: weeks' to attend the launch of Vox's European election campaign, newspaper El Pais reported. Mr Mile...(truncated)
chunk_index: 3
description: A row breaks out after Spain's transport minister suggests Argentina's president has taken drugs.
link: https://www.bbc.com/news/articles/czd8qzvpl4lo
pubDate: 2024-05-04 15:56:45+00:00
title: Spain-Argentina row over drug-use accusation

article_content: Opposition supporters have gathered across Venezuela to protest against Nicolás Maduro's disputed vi...(truncated)
chunk: the world, from Australia to Spain and also in the United Kingdom, Canada, Colombia, Mexico and Arge...(truncated)
chunk_index: 4
description: Opposition leader María Corina Machado joined thousands of demonstrators in the capital Caracas.
link: https://www.bbc.com/news/articles/cgedgqqy7x9o
pubDate: 2024-08-17 23:19:48+00:00
title: Protests across Venezuela as election dispute goes on
```

In [8]:
# Test your function!
unittests.test_semantic_search_with_reranking(semantic_search_with_reranking, client)

 All tests passed!


<a id='4'></a>
## 4 - Incorporating the Weaviate API into our previous schema
---

This section is not graded. Here, you will revisit the functions used throughout the assignments to integrate the Weaviate API into your existing schema. Once integrated, you will be able to run prompts and test your new RAG system!

<a id='4-1'></a>
### 4.1 Generating the final prompt


In [17]:
def generate_final_prompt(
    query: str,
    top_k: int,
    retrieve_function: Callable,
    rerank_query: Optional[str] = None,
    rerank_property: Optional[str] = None,
    use_rerank: bool = False,
    use_rag: bool = True,
) -> str:
    """
    Generates a final prompt by optionally retrieving and formatting relevant documents using retrieval-augmented generation (RAG).

    Args:
        query (str): The initial query to be used for document retrieval.
        top_k (int): The number of top documents to retrieve and use for generating the prompt.
        retrieve_function (callable): The function used to retrieve documents based on the query.
        rerank_query (str, optional): The query used specifically for reranking documents if reranking is enabled.
        rerank_property (str, optional): The property used for reranking. Required if 'use_rerank' is True.
        use_rerank (bool, optional): Flag to denote whether to use reranking in document retrieval. Defaults to False.
        use_rag (bool, optional): Flag to determine whether to use retrieval-augmented generation. Defaults to True.

    Returns:
        str: A constructed prompt that includes the original query and formatted retrieved documents if 'use_rag' is True.
             Otherwise, it returns the original query.
    """
    # If no rag, return the query
    if not use_rag:
        return query

    if use_rerank:
        if rerank_property is None:
            raise ValueError("rerank_property must be set if use_rerank = True")
        top_k_documents = retrieve_function(
            query=query,
            top_k=top_k,
            collection=collection,
            rerank_property=rerank_property,
            rerank_query=rerank_query,
        )
    else:
        top_k_documents = retrieve_function(
            query=query, top_k=top_k, collection=collection
        )

    # Initialize an empty string to store the formatted data.
    formatted_data = ""

    # Iterate over each retrieved document.
    for document in top_k_documents:
        # Format each document into a structured string.
        document_layout = (
            f"Title: {document['title']}, Chunk: {document['chunk']}, "
            f"Published at: {document['pubDate']}\nURL: {document['link']}"
        )
        # Append the formatted string to the main data string with a newline for separation.
        formatted_data += document_layout + "\n"

    # If use_rag flag is True, construct the enhanced prompt with the augmented data.
    retrieve_data_formatted = formatted_data  # Store formatted data.
    prompt = (
        f"Answer the user query below. There will be provided additional information for you to compose your answer. "
        f"The relevant information provided is from 2024 and it should be added as your overall knowledge to answer the query, "
        f"you should not rely only on this information to answer the query, but add it to your overall knowledge."
        f"The news data is ordered by relevance.\n"
        f"Query: {query}\n"
        f"2024 News: {retrieve_data_formatted}"
    )

    return prompt

In [18]:
prompt = generate_final_prompt(
    "Tell me the economic situation of the US in 2024.",
    top_k=5,
    retrieve_function=semantic_search_retrieve,
    use_rerank=False,
    rerank_property="title",
)

In [19]:
print(prompt)

Answer the user query below. There will be provided additional information for you to compose your answer. The relevant information provided is from 2024 and it should be added as your overall knowledge to answer the query, you should not rely only on this information to answer the query, but add it to your overall knowledge.The news data is ordered by relevance.
Query: Tell me the economic situation of the US in 2024.
2024 News: Title: What is a recession and how could one affect me?, Chunk: which was much better than expected. That put the US at 2.5% over 2023 as a whole, the best performance of all other advanced economies. It is also expected to outperform the rest of the G7 in 2024. In October 2023, the International Monetary Fund (IMF) predicted that the UK would grow by just 0.6% in 2024. The independent Office for Budget Responsibility (OBR) expects the UK economy to grow by 0.7% in 2024, but that is less than half of its earlier prediction of 1.8% growth. How will the UK econo

<a id='4-2'></a>
### 4.2 LLM call

Let's revisit the `llm_call` function, now adapted to this assignment.

In [20]:
def llm_call(
    query: str,
    retrieve_function: Optional[Callable],
    top_k: int = 5,
    use_rag: bool = True,
    use_rerank: bool = False,
    rerank_property: Optional[str] = None,
    rerank_query: Optional[str] = None,
) -> str:
    """
    Simulates a call to a language model by generating a prompt and using it to produce a response.

    Args:
        query (str): The initial query for which a response is sought.
        retrieve_function (callable, optional): The function used to retrieve documents related to the query.
        top_k (int, optional): The number of top documents to retrieve and use for generating the prompt. Defaults to 5.
        use_rag (bool, optional): Indicates whether to use retrieval-augmented generation. Defaults to True.
        use_rerank (bool, optional): Indicates whether to apply reranking to the retrieved documents. Defaults to False.
        rerank_property (str, optional): The property to use for reranking. Required if 'use_rerank' is True.
        rerank_query (str, optional): The query used specifically for reranking documents if reranking is enabled.

    Returns:
        str: The generated response content after processing the prompt with a language model.
    """

    # Get the prompt
    PROMPT = generate_final_prompt(
        query,
        top_k=top_k,
        retrieve_function=retrieve_function,
        use_rag=use_rag,
        use_rerank=use_rerank,
        rerank_property=rerank_property,
        rerank_query=rerank_query,
    )

    generated_response = generate_with_single_input(PROMPT)

    generated_message = generated_response["content"]

    return generated_message

In [0]:
query = "Tell me about United States and Brazil's relationship over the course of 2024. Provide links for the resources you use in the answer."

In [23]:
# Result with reranked results
print(
    llm_call(
        query=query,
        top_k=5,
        retrieve_function=hybrid_retrieve,
    )
)

The conflicts in Latin America involve various issues that affect several countries in the region. Some of the key conflicts and issues that have been reported include:

1. Gang violence in Ecuador: Ecuador has been plagued by high levels of gang violence, resulting in a significant increase in violent deaths. The country has one of the highest murder rates in Latin America, with police recording around 8,000 violent deaths in 2023. This has made Ecuador an increasingly unsafe destination for tourists, who are starting to avoid the country.

2. Election dispute in Venezuela: The vote for Venezuela's president, which took place in 2024, has been marred by controversy and disputes. The winner of the election is not recognized by the European Union, the United States, or several other Latin American countries.

3. Rising female leadership: The role of women in Latin America's politics is transforming rapidly. As many countries head towards the polls, women are increasingly taking on promi

<a id='5'></a>
## 5 - Experimenting with Your RAG System

Now it is time for you to experiment with the system! Run the next cell to load a widget that will input a query, a rerank property, and output five different LLM responses:

1. With semantic search
2. With semantic search and reranking
3. With BM25 search
4. With hybrid search
5. Without RAG


In [26]:
display_widget(
    llm_call,
    semantic_search_retrieve,
    bm25_retrieve,
    hybrid_retrieve,
    semantic_search_with_reranking,
)

c:\Users\91811\AppData\Local\pypoetry\Cache\virtualenvs\rag-kkX-x6bK-py3.13\Lib\site-packages\traitlets\traitlets.py:1385: DeprecationWarning: Passing unrecognized arguments to super(Layout).__init__(align='center').
object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  warn(
c:\Users\91811\AppData\Local\pypoetry\Cache\virtualenvs\rag-kkX-x6bK-py3.13\Lib\site-packages\traitlets\traitlets.py:1385: DeprecationWarning: Passing unrecognized arguments to super(ButtonStyle).__init__(font_color='black').
object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  warn(


HTML(value='\n    <style>\n        .custom-output {\n            background-color: #f9f9f9;\n            color…

IntSlider(value=5, description='Top K:', max=20, min=1, style=SliderStyle(description_width='initial'))

Dropdown(description='Rerank Property:', options=('title', 'chunk'), style=DescriptionStyle(description_width=…

Button(description='Get Responses', style=ButtonStyle(button_color='#eee'))

Output()

Congratulations on finishing this assignment!